In [1]:
import pandas as pd
import os
import dask
from dask.diagnostics import ProgressBar
import utils

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

0it [00:00, ?it/s]


In [2]:
csv_ls = [os.path.join('chunk_data/',i) for i in os.listdir('chunk_data/') if ('.csv' in i) and ('-' in i)]

In [4]:
container = []

for i in csv_ls : 
    load = dask.delayed(pd.read_csv)(i)
    container.append(load)
concat_df = dask.delayed(pd.concat)(container,ignore_index=True)

In [5]:
with ProgressBar() : 
    df = concat_df.compute()

[########################################] | 100% Completed |  4.1s


In [6]:
df.columns = ['path','text']

In [9]:
def finer_lemmatize(word) : 
    word = lemmatizer.lemmatize(word,'n')
    word = lemmatizer.lemmatize(word,'v')
    return word

In [10]:
df['lemmatized_text'] = \
df.text.str.split(" ").progress_apply(lambda sent : ','.join(list(map(finer_lemmatize,sent))).replace(",",' '))

954931it [09:52, 1612.08it/s]


In [11]:
df.to_csv('data/test_summarized_with_path.csv')

___________________

In [12]:
# df = pd.read_csv('data/summarized_with_path.csv')
result_df = pd.DataFrame()
chunk_ls = utils.make_chunk(df.index.tolist(),10)

In [14]:
reload(utils)

<module 'utils' from 'utils.pyc'>

In [15]:
for idx,index in enumerate(chunk_ls) : 
    tmp = df.iloc[index]
#     print("{} th dataframe is manipulated".format(idx),end='  ')
    %time tmp['parsed_text'] = tmp.lemmatized_text.\
    progress_apply(utils.parsing)
#     print('{} th parsing is finished'.format(idx),end='  ')
    tmp['length'] = tmp.parsed_text.apply(lambda x : len(x.split(" ")))
    use_tmp = tmp[tmp.length <= 100].reset_index(drop=True)
    result_df = pd.concat([result_df,use_tmp],ignore_index=True)
    print(result_df.shape)

100%|██████████| 95493/95493 [00:19<00:00, 4941.29it/s]
/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


CPU times: user 18.4 s, sys: 588 ms, total: 19 s
Wall time: 19.5 s


/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  1%|          | 509/95493 [00:00<00:18, 5005.85it/s]

(65316, 5)


100%|██████████| 95493/95493 [00:16<00:00, 5921.07it/s]


...

CPU times: user 15.9 s, sys: 764 ms, total: 16.6 s
Wall time: 16.9 s


  0%|          | 392/95493 [00:00<00:24, 3911.40it/s]

(588326, 5)


100%|██████████| 95493/95493 [00:16<00:00, 5959.27it/s]


CPU times: user 15.4 s, sys: 540 ms, total: 16 s
Wall time: 16.2 s
(661147, 5)


In [16]:
result_df.to_csv("data/test_result_df.csv",index=False)